In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import json
import tensorflowjs as tfjs

## Import Text

In [2]:
with open("./trainingData/sherlockS.txt", 'r', encoding='utf-8') as file:
    text = file.read()

## Tokenize Text

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [10]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

print(input_sequences[:20])

[[268, 61], [268, 61, 19], [268, 61, 19, 93], [268, 61, 19, 93, 14], [268, 61, 19, 93, 14, 94], [268, 61, 19, 93, 14, 94, 1], [268, 61, 19, 93, 14, 94, 1, 95], [268, 61, 19, 93, 14, 94, 1, 95, 6], [18, 145], [18, 145, 96], [18, 145, 96, 62], [18, 145, 96, 62, 269], [18, 145, 96, 62, 269, 97], [18, 145, 96, 62, 269, 97, 146], [18, 145, 96, 62, 269, 97, 146, 147], [98, 148], [98, 148, 10], [98, 148, 10, 7], [98, 148, 10, 7, 63], [98, 148, 10, 7, 63, 93]]


In [11]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

print(input_sequences[:8])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0 268  61]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0 268  61  19]
 [  0   0   0   0   0   0   0   0   0   0   0   0 268  61  19  93]
 [  0   0   0   0   0   0   0   0   0   0   0 268  61  19  93  14]
 [  0   0   0   0   0   0   0   0   0   0 268  61  19  93  14  94]
 [  0   0   0   0   0   0   0   0   0 268  61  19  93  14  94   1]
 [  0   0   0   0   0   0   0   0 268  61  19  93  14  94   1  95]]


In [8]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

print(X[2])
print(y[2])

[  0   0   0   0   0   0   0   0   0   0   0   0 268  61  19]
93


In [7]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

## Create Model Architecture

In [8]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           935900    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 9359)              1413209   
                                                                 
Total params: 2499709 (9.54 MB)
Trainable params: 2499709 (9.54 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


## Train Model

In [9]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100


: 

: 

## Save Model for JS

In [ ]:

output_dir = "./tfjs_models/RNN" 
tfjs.converters.save_keras_model(model, output_dir)

## Save word index

In [ ]:
wordIndex = {}
for word in tokenizer.word_index:
    wordIndex[word] = tokenizer.word_index[word]

with open('./tokenizer/wordIndex.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(wordIndex, ensure_ascii=False))

## Save Max Sequence Length

In [ ]:
with open('./tokenizer/max_sequence_len.txt', 'w') as f:
    f.write(str(max_sequence_len))